In [12]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Import the data

### FAOSTAT_data

In [16]:
faostat_df = pd.read_csv("/kaggle/input/temperature-change/FAOSTAT_data_11-24-2020.csv")

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/temperature-change/FAOSTAT_data_11-24-2020.csv'

In [17]:
faostat_df.head(10)

NameError: name 'faostat_df' is not defined

### Environment Temperature change

In [18]:
envtem_df = pd.read_csv("/kaggle/input/temperature-change/Environment_Temperature_change_E_All_Data_NOFLAG.csv", encoding = 'latin1')

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/temperature-change/Environment_Temperature_change_E_All_Data_NOFLAG.csv'

In [8]:
envtem_df.head(5)

NameError: name 'envtem_df' is not defined

## Feature selection

In [9]:
#Get a summary of the data
envtem_df.info()

NameError: name 'envtem_df' is not defined

In [10]:
#View the dimensions of the dataframe
envtem_df.shape

NameError: name 'envtem_df' is not defined

In [8]:
#get only where Element is Temperature change
envtem_df = envtem_df[envtem_df['Element'] == 'Temperature change']

In [9]:
#Drop columns 'Area Code', 'Months Code', 'Element Code', 'Unit'
envtem_df.drop(columns=['Area Code', 'Months Code', 'Element Code','Element', 'Unit'], axis=1, inplace=True)
envtem_df.head()

,Area,Months,Y1961,Y1962,Y1963,Y1964,Y1965,Y1966,Y1967,Y1968,...,Y2010,Y2011,Y2012,Y2013,Y2014,Y2015,Y2016,Y2017,Y2018,Y2019
0,Afghanistan,January,0.777,0.062,2.744,-5.232,1.868,3.629,-1.432,0.389,...,3.601,1.179,-0.583,1.233,1.755,1.943,3.416,1.201,1.996,2.951
2,Afghanistan,February,-1.743,2.465,3.919,-0.202,-0.096,3.397,0.296,-2.055,...,1.212,0.321,-3.201,1.494,-3.187,2.699,2.251,-0.323,2.705,0.086
4,Afghanistan,March,0.516,1.336,0.403,1.659,-0.909,-0.069,-0.759,0.496,...,3.390,0.748,-0.527,2.246,-0.076,-0.497,2.296,0.834,4.418,0.234
6,Afghanistan,April,-1.709,0.117,0.919,-0.533,-1.816,-1.192,-1.496,-0.590,...,2.591,1.712,1.417,-0.052,0.585,1.589,0.980,1.252,1.442,0.899
8,Afghanistan,May,1.412,-0.092,-0.690,-0.160,-0.190,-0.192,-1.307,-1.841,...,1.419,3.643,0.909,1.201,0.959,1.862,3.246,3.280,0.855,0.647


In [10]:
#Get the unique values of columns Months
envtem_df['Months'].unique()

array(['January', 'February', 'March', 'April', 'May', 'June', 'July',
       'August', 'September', 'October', 'November', 'December',
       'Dec\x96Jan\x96Feb', 'Mar\x96Apr\x96May', 'Jun\x96Jul\x96Aug',
       'Sep\x96Oct\x96Nov', 'Meteorological year'], dtype=object)

In [11]:
#Get values where months is in : 'January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December'
envtem_df = envtem_df[envtem_df['Months'].isin(['January', 'February', 'March', 'April', 'May', 'June', 'July',
       'August', 'September', 'October', 'November', 'December'])]
envtem_df['Months'].unique()

array(['January', 'February', 'March', 'April', 'May', 'June', 'July',
       'August', 'September', 'October', 'November', 'December'],
      dtype=object)

In [12]:
#Get the columns dataframe
envtem_df.columns

Index(['Area', 'Months', 'Y1961', 'Y1962', 'Y1963', 'Y1964', 'Y1965', 'Y1966',
       'Y1967', 'Y1968', 'Y1969', 'Y1970', 'Y1971', 'Y1972', 'Y1973', 'Y1974',
       'Y1975', 'Y1976', 'Y1977', 'Y1978', 'Y1979', 'Y1980', 'Y1981', 'Y1982',
       'Y1983', 'Y1984', 'Y1985', 'Y1986', 'Y1987', 'Y1988', 'Y1989', 'Y1990',
       'Y1991', 'Y1992', 'Y1993', 'Y1994', 'Y1995', 'Y1996', 'Y1997', 'Y1998',
       'Y1999', 'Y2000', 'Y2001', 'Y2002', 'Y2003', 'Y2004', 'Y2005', 'Y2006',
       'Y2007', 'Y2008', 'Y2009', 'Y2010', 'Y2011', 'Y2012', 'Y2013', 'Y2014',
       'Y2015', 'Y2016', 'Y2017', 'Y2018', 'Y2019'],
      dtype='object')

In [13]:
#Turn years columns into rows using melt function
envtem_df = pd.melt(envtem_df, id_vars = ['Area', 'Months'], var_name = 'years', value_name = 'temperature')
envtem_df.head()

,Area,Months,years,temperature
0,Afghanistan,January,Y1961,0.777
1,Afghanistan,February,Y1961,-1.743
2,Afghanistan,March,Y1961,0.516
3,Afghanistan,April,Y1961,-1.709
4,Afghanistan,May,Y1961,1.412


In [14]:
#years without the Y letter
envtem_df['years'] = envtem_df['years'].str[1:]
envtem_df.head()

,Area,Months,years,temperature
0,Afghanistan,January,1961,0.777
1,Afghanistan,February,1961,-1.743
2,Afghanistan,March,1961,0.516
3,Afghanistan,April,1961,-1.709
4,Afghanistan,May,1961,1.412


In [2]:
#Change the dtype from object to str
envtem_df[['Area', 'Months', 'years']] = envtem_df[['Area', 'Months', 'years']].astype('str')

NameError: name 'envtem_df' is not defined

# World temperature

In [1]:
envtem_df['Area'].unique()

NameError: name 'envtem_df' is not defined

In [17]:
#Create a wolrd temperature dataframe
world_df = envtem_df.loc[envtem_df.Area == 'World']

#Grouping by years to get the average of temperature in each year
grp_wolrd_df = world_df.groupby(['years'], as_index = False).mean()
grp_wolrd_df.head()

,years,temperature
0,1961,0.134583
1,1962,0.061333
2,1963,0.130750
3,1964,-0.287583
4,1965,-0.148167


In [18]:
fig = px.line(grp_wolrd_df, x="years", y="temperature", title='Tempereature of the wolrd from 1961 to 2019', color_discrete_map = {"temperature":"red"})
fig.show()

In [19]:
fig = px.bar(grp_wolrd_df, x="years", y="temperature", title='Tempereature of the wolrd from 1961 to 2019', color_discrete_map = {"temperature":"red"})
fig.show()

In [20]:
wor_6119_df = world_df[world_df['years'].isin(['1961', '1971', '1981', '1991', '2001', '2019'])]
wor_6119_df.head()

,Area,Months,years,temperature
2964,World,January,1961,0.399
2965,World,February,1961,0.263
2966,World,March,1961,0.254
2967,World,April,1961,0.278
2968,World,May,1961,0.367


In [21]:
fig = px.line(wor_6119_df, x="Months", y="temperature", color='years', title='Comparing tempereature of the wolrd')
fig.show()

# France

In [22]:
#Create France temperature dataframe
France_df = envtem_df.loc[envtem_df.Area == 'France']

#Grouping by years to get the average of temperature in each year
grp_Fr_df = France_df.groupby(['years'], as_index = False).mean()
grp_Fr_df.head()

,years,temperature
0,1961,0.956667
1,1962,-0.560250
2,1963,-0.939333
3,1964,0.160833
4,1965,-0.418750


In [23]:
#check if there is missing data
missing = France_df.isnull()
for col in missing:
    print(col)
    print(missing[col].value_counts())

Area
False    708
Name: Area, dtype: int64
Months
False    708
Name: Months, dtype: int64
years
False    708
Name: years, dtype: int64
temperature
False    708
Name: temperature, dtype: int64


In [24]:
fig = px.bar(grp_Fr_df, x="years", y="temperature", title='Tempereature in the France from 1961 to 2019', color_discrete_map = {"temperature":"red"})
fig.show()

In [25]:
#Create a data frame for years 1961, 1971, 1981, 1991, 2001, 2019
fr_6119_df = France_df[France_df['years'].isin(['1961', '1971', '1981', '1991', '2001', '2019'])]
fr_6119_df.head()

,Area,Months,years,temperature
960,France,January,1961,0.098
961,France,February,1961,3.417
962,France,March,1961,1.726
963,France,April,1961,2.680
964,France,May,1961,-0.249


In [26]:
fig = px.line(fr_6119_df, x="Months", y="temperature", color='years', title='Comparing tempereature in France')
fig.show()

# Algeria

In [27]:
#Create Algeria temperature dataframe
Algeria_df = envtem_df.loc[envtem_df.Area == 'Algeria']

#Grouping by years to get the average of temperature in each year
grp_alg_df = Algeria_df.groupby(['years'], as_index = False).mean()
grp_alg_df.head()

,years,temperature
0,1961,3.504167e-01
1,1962,-1.387779e-17
2,1963,2.317500e-01
3,1964,-8.250000e-03
4,1965,-3.191667e-02


In [28]:
#check if there is missing data
missing = Algeria_df.isnull()
for col in missing:
    print(col)
    print(missing[col].value_counts())

Area
False    708
Name: Area, dtype: int64
Months
False    708
Name: Months, dtype: int64
years
False    708
Name: years, dtype: int64
temperature
False    708
Name: temperature, dtype: int64


In [29]:
fig = px.bar(grp_alg_df, x="years", y="temperature", labels={'x':'Years', 'y':'Temperature'}, title='Tempereature in the Algeria from 1961 to 2019', color_discrete_map = {"temperature":"red"})
fig.show()

In [30]:
alg_6119_df = Algeria_df[Algeria_df['years'].isin(['1961', '1971', '1981', '1991', '2001', '2019'])]
alg_6119_df.head()

,Area,Months,years,temperature
24,Algeria,January,1961,-1.022
25,Algeria,February,1961,0.196
26,Algeria,March,1961,-1.051
27,Algeria,April,1961,1.720
28,Algeria,May,1961,1.863


In [31]:
fig = px.line(alg_6119_df, x="Months", y="temperature", color='years', title='Comparing tempereature in Algeria')
fig.show()